<a href="https://colab.research.google.com/github/jihyeon602/DeepLearning_practice/blob/main/%EB%B0%98%EB%A0%A4%EB%8F%99%EB%AC%BC_%EC%A7%88%EB%B3%91_%EC%A7%84%EB%8B%A8%EC%9D%84_%EC%9C%84%ED%95%9C_%EC%98%81%EC%83%81_%EB%8D%B0%EC%9D%B4%ED%84%B0_(%EA%B7%BC%EA%B3%A8%EA%B2%A9%EA%B3%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 파일 로드

In [ ]:
!pip install -U gdown

!gdown 11SIkR8k0sUVKF9t73HcVo5Ib_rNjLLWD --output VS1_CAT.zip

In [11]:
!file TS1_CAT.zip

TS2_DOG.zip: Zip archive data, at least v2.0 to extract, compression method=deflate


In [14]:
import zipfile

zip_file_path = '/content/VS1_CAT.zip'
extract_to = '/content/val/VS1_CAT'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

압축 해제 완료!


In [18]:
zip_file_path = '/content/TL1_CAT.zip'
extract_to = '/content/train/T_lavels'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

압축 해제 완료!


# 고양이 질병 훈련

### json txt/csv 파일변환

In [19]:
import os
import json

input_folder = '/content/train/T_lavels'
output_file = '/content/train/t_lavel.txt'

with open(output_file, 'w', encoding='utf-8') as out_file:
    for filename in sorted(os.listdir(input_folder)):
        if filename.endswith('.json'):
            file_path = os.path.join(input_folder, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                out_file.write(json.dumps(data, ensure_ascii=False) + '\n')


In [20]:
import pandas as pd

txt_file_path = "/content/train/t_lavel.txt"

records = []
with open(txt_file_path, "r", encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line)
        metadata = entry.get("metadata", {})
        record = {
            "Patient-id": metadata.get("Birth", ""),
            "Annotation-id": metadata.get("Annotation-id", ""),
            "Sex": metadata.get("Sex", ""),
            "Date": metadata.get("Date", ""),
            "Position": metadata.get("Position", ""),
            "Disease": metadata.get("Disease", ""),
            "Disease-Name": metadata.get("Disease-Name", ""),
            "fileName": entry.get("fileName", "")
        }
        records.append(record)

# 정렬
t_lavel_df = pd.DataFrame(records)
t_lavel_df = t_lavel_df.sort_values(by=["Patient-id"]).reset_index(drop=True)

# CSV 저장
csv_path = "/content/train/t_lavel.csv"
t_lavel_df.to_csv(csv_path, index=False)


### 라벨링된 JSON 파일을 YOLO 포맷 텍스트 파일로 변환

- train/val 각각 result_labels 폴더에 저장됨

In [26]:
import os
import json
from PIL import Image

# 기본 경로
base_path = "/content/"

# 경로 딕셔너리
sets = {
    'train': {
        'json_folder': os.path.join(base_path, "train", "T_lavels"),
        'image_folder': os.path.join(base_path, "train", "TS1_CAT"),
        'output_folder': os.path.join(base_path, "train", "result_labels")
    },
    'val': {
        'json_folder': os.path.join(base_path, "val", "V_lavels"),
        'image_folder': os.path.join(base_path, "val", "VS1_CAT"),
        'output_folder': os.path.join(base_path, "val", "result_labels")
    }
}

# 지원 이미지 확장자
image_exts = ['.jpg', '.jpeg', '.png']

# 변환 함수 정의
def convert_to_yolo(json_folder, image_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for file in os.listdir(json_folder):
        if file.endswith('.json'):
            base_name = os.path.splitext(file)[0]

            # JSON 로드
            with open(os.path.join(json_folder, file), 'r', encoding='utf-8') as f:
                data = json.load(f)

            # 이미지 찾기
            image_path = None
            for ext in image_exts:
                candidate = os.path.join(image_folder, base_name + ext)
                if os.path.exists(candidate):
                    image_path = candidate
                    break

            if not image_path:
                print(f"⚠️ 이미지 없음: {base_name}")
                continue

            # 이미지 크기
            with Image.open(image_path) as img:
                img_width, img_height = img.size

            yolo_lines = []
            for ann in data.get("annotations", []):
                if ann.get("shape") == "Bounding Box":
                    (x1, y1), (x2, y2) = ann["points"]
                    cx = (x1 + x2) / 2 / img_width
                    cy = (y1 + y2) / 2 / img_height
                    w = abs(x2 - x1) / img_width
                    h = abs(y2 - y1) / img_height
                    yolo_lines.append(f"0 {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

            # 결과 저장
            out_path = os.path.join(output_folder, base_name + ".txt")
            with open(out_path, "w", encoding='utf-8') as f:
                f.write("\n".join(yolo_lines))

            print(f"✅ 변환 완료: {base_name}.txt")

# ▶ 각 세트(train, val)에 대해 실행
for phase, paths in sets.items():
    print(f"\n📂 처리 중: {phase}")
    convert_to_yolo(paths['json_folder'], paths['image_folder'], paths['output_folder'])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
✅ 변환 완료: C_62_20100808_IF_0030_NOR_Mu07_20211224_0340.txt
✅ 변환 완료: C_62_20180930_SF_0032_NOR_Mu06_20211224_0782.txt
✅ 변환 완료: C_62_20130106_CM_0020_NOR_Mu04_20211210_0141.txt
✅ 변환 완료: C_62_20110104_IM_0032_NOR_Mu03_20170828_0002.txt
✅ 변환 완료: C_62_20020128_IF_0009_NOR_Mu02_20090112_0001.txt
✅ 변환 완료: C_62_20081209_CM_0020_NOR_Mu07_20211224_0191.txt
✅ 변환 완료: C_62_20090722_IF_0020_NOR_Mu03_20211213_0149.txt
✅ 변환 완료: C_62_20180913_IM_0009_NOR_Mu02_20190325_0123.txt
✅ 변환 완료: C_62_20180912_IM_0009_NOR_Mu03_20190213_0327.txt
✅ 변환 완료: C_62_20120505_IF_0009_NOR_Mu03_20190402_0144.txt
✅ 변환 완료: C_62_20160622_IF_0032_NOR_Mu07_20211224_1579.txt
✅ 변환 완료: C_62_20130727_SF_0032_NOR_Mu03_20211221_0321.txt
✅ 변환 완료: C_62_20041207_IM_0009_NOR_Mu03_20090108_0001.txt
✅ 변환 완료: C_62_20180811_IM_0009_NOR_Mu03_20181111_0321.txt
✅ 변환 완료: C_62_20040513_IF_0032_NOR_Mu07_20120125_0001.txt
✅ 변환 완료: C_62_20090328_CM_0020_NOR_Mu06_20211224_0072.txt
✅ 변환 완료: C_62_20070317_SF_0009_NOR_M

In [34]:
!mv /content/CAT/images/train/TS1_CAT/* /content/CAT/images/train

In [35]:
!mv /content/CAT/images/val/VS1_CAT/* /content/CAT/images/val

In [36]:
!mv /content/CAT/labels/train/result_labels/* /content/CAT/labels/train

In [37]:
!mv /content/CAT/labels/val/result_labels/*  /content/CAT/labels/val

- yaml 파일 제작

In [38]:
# YAML 파일 내용
yaml_content = """
train: /content/CAT/images/train
val: /content/CAT/images/val

nc: 1
names: ['cat']
"""

# 'data.yaml' 파일을 지정된 경로에 생성
with open('/content/CAT/cat.yaml', 'w') as file:
    file.write(yaml_content)

print("cat.yaml 파일 생성완료.")

cat.yaml 파일 생성완료.


## yolo11s 사용

In [28]:
!git clone https://github.com/ultralytics/yolo11

Cloning into 'yolo11'...
fatal: could not read Username for 'https://github.com': No such device or address


In [32]:
!pip install ultralytics

In [40]:
!yolo detect train data='/content/CAT/cat.yaml' model=yolo11m.pt epochs=50 imgsz=640 patience=5

Ultralytics 8.3.116 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/CAT/cat.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line

In [43]:
!rm -rf /content/runs/detect/train3

In [ ]:
#모델 학습 과정 확인
%load_ext tensorboard
%tensorboard --logdir /content/runs/detect/train4